<a href="https://colab.research.google.com/github/thedataninja1786/Machine-Learning/blob/main/CNN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision 
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt 
import numpy as np 
from google.colab import drive 
drive.mount('/content/drive')

In [3]:
#Hyperparameters 
epochs = 5 
batch_size = 4 
learning_rate = 0.001 

In [10]:
#dataset has PILImage images of range [0,1]
#Transorm them to tensors of normalized range [-1,1]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [ ]:
# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class

train_dataset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/CIFAR10', train = True, download = True, transform = transform)

test_dataset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/CIFAR10', train = False, download = True, transform = transform)
 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [31]:
#Create the CNN 
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    '''nout = [width + 2padding - kernel_size) / stride] + 1 '''
    # [4,3,32,32]
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 5) # in channels is 3 since we are using colored imgs, out_channels = outputs 6 convolutional features (selected at random) 
    # [4,6,28,28]
    self.pool1 = nn.MaxPool2d(2,2) #reduces the images by a factor of 2
    # [4,6,14,14]
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5)
    # [4,16,10,10]
    self.pool2 = nn.MaxPool2d(2,2)
    # [4,16,5,5] which becomes the input of the fully connected layer 
    self.fc1 = nn.Linear(in_features = (16 * 5 * 5), out_features = 120) # (120 neurons selected at random)
    self.fc2 = nn.Linear(in_features = 120, out_features = 84) # 84 selected at random 
    self.fc3 = nn.Linear(in_features = 84, out_features = 10) #final layer, output will be 10 same as the number of classes 

  def forward(self, x):
    # where n refers to the batch_size 
    # -> n, 3, 32, 32
    x = self.pool1(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
    x = self.pool2(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
    x = x.view(-1, 16 * 5 * 5)            # -> n, 400
    x = F.relu(self.fc1(x))               # -> n, 120
    x = F.relu(self.fc2(x))               # -> n, 84
    x = self.fc3(x)                       # -> n, 10
    return x

In [ ]:
model = CNN()
loss_function = nn.CrossEntropyLoss() #includes the softmax activation function 
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

n_total_steps = len(train_loader)
for epoch in range(epochs):
  n_correct = 0
  n_samples = 0
  for i, (images, labels) in enumerate(train_loader):
    # Forward pass
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    loss = loss_function(outputs, labels)
    # Backpropagation and optimization 
    optimizer.zero_grad() #empty gradients 
    loss.backward()
    optimizer.step()

    acc = 100.0 * n_correct / n_samples

    if (i+1) % 2000 == 0:
      print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], Accuracy: {round(acc,2)} %, Loss: {loss.item():.4f}')
print('Done!!')



In [42]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    outputs = model(images)
    # max returns (value ,index)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item() 
    
    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
          n_class_correct[label] += 1 #counts correct answers 
      n_class_samples[label] += 1 #counts occurrences of each class 

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc} %')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of the network: 47.83 %
Accuracy of plane: 55.7 %
Accuracy of car: 55.9 %
Accuracy of bird: 26.7 %
Accuracy of cat: 18.2 %
Accuracy of deer: 37.7 %
Accuracy of dog: 43.6 %
Accuracy of frog: 65.4 %
Accuracy of horse: 63.3 %
Accuracy of ship: 50.4 %
Accuracy of truck: 61.4 %
